# NHL Data Exploration Notebook

This notebook provides a simple interface to explore and analyze NHL data stored in your MySQL database. You can modify queries and visualizations to explore different aspects of the data.

## Overview
- Connect to your MySQL database
- Explore database structure
- Run custom queries
- Visualize data trends
- Perform basic statistical analysis

## 1. Import Required Libraries

Let's start by importing all the libraries we'll need for data exploration and visualization.

In [2]:
# Import required libraries
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from config import db_config
from plotly.subplots import make_subplots
import warnings
import ipywidgets as widgets
from IPython.display import display
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

# Set plotting style (using default instead of seaborn-v0_8 to avoid crash)
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Connect to MySQL Database

Update the connection parameters below with your database credentials.

In [3]:
# Establish connection
connection = mysql.connector.connect(
    host=db_config["host"],
    user=db_config["user"],
    password=db_config["password"],
    database=db_config["database"]
)
cursor = connection.cursor()

In [4]:
teams_dict = {
    'team_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 46, 47, 48, 49, 52, 53, 54, 55, 56, 59],
    'team_name': ['New Jersey Devils', 'New York Islanders', 'New York Rangers', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins', 'Buffalo Sabres', 'Montreal Canadiens', 'Ottawa Senators', 'Toronto Maple Leafs', 'Atlanta Thrashers', 'Carolina Hurricanes', 'Florida Panthers', 'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks', 'Detroit Red Wings', 'Nashville Predators', 'St Louis Blues', 'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers', 'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars', 'Los Angeles Kings', 'Pheonix Coyotes', 'San Jose Sharks', 'Columbus Blue Jackets', 'Minnesota Wild', 'Minnesota North Stars', 'Quebec Nordique', 'Winnipeg Jets (1979)', 'Hartford Whalers', 'Colorado Rockies', 'Oakland Seals', 'Atlanta Flames', 'Kansas City Scouts', 'Cleveland Barons', 'Winnipeg Jets', 'Arizona Coyotes', 'Vegas Golden Knights', 'Seattle Kraken', 'California Golden Seals', 'Utah Hockey Club'],
    'team_abbreviation': ['NJD', 'NYI', 'NYR', 'PHI', 'PIT', 'BOS', 'BUF', 'MTL', 'OTT', 'TOR', 'ATL', 'CAR', 'FLA', 'TBL', 'WSH', 'CHI', 'DET', 'NSH', 'STL', 'CGY', 'COL', 'EDM', 'VAN', 'ANA', 'DAL', 'LAK', 'PHX', 'SJS', 'CBJ', 'MIN', 'MNS', 'QUE', 'WIN', 'HFD', 'CLR', 'OAK', 'AFM', 'KCS', 'CLE', 'WPG', 'ARI', 'VGK', 'SEA', 'CGS', 'UTA']
}

## 3. Explore Database Structure

Let's see what tables and data we have available in the database.

In [5]:
# Show all tables in the database
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

print("Available tables:")
table_names = []
for table in tables:
    table_name = table[0]
    table_names.append(table_name)
    print(f"- {table_name}")

print(f"\nTotal tables: {len(table_names)}")

# Function to easily query database and return DataFrame
def query_to_df(query, params=None):
    """Execute query and return results as pandas DataFrame"""
    try:
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        
        # Get column names
        columns = [desc[0] for desc in cursor.description]
        
        # Fetch data and create DataFrame
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except Exception as e:
        print(f"Query error: {e}")
        return pd.DataFrame()

Available tables:
- events
- games
- players
- players_season
- roster_players
- seasons
- seasons_end_standings
- teams

Total tables: 8


In [6]:
# Explore table structures (modify table name as needed)
def explore_table_structure(table_name):
    """Show structure and sample data for a table"""
    print(f"\n=== Table: {table_name} ===")
    
    # Show table structure
    cursor.execute(f"DESCRIBE {table_name}")
    structure = cursor.fetchall()
    
    print("Columns:")
    for col in structure:
        print(f"  {col[0]} ({col[1]}) - {col[2]} - {col[3]}")
    
    # Show row count
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    print(f"\nTotal rows: {row_count:,}")
    
    # Show sample data
    sample_df = query_to_df(f"SELECT * FROM {table_name} LIMIT 5")
    print(f"\nSample data:")
    return sample_df

# Explore the first table (modify index or table name as needed)
if table_names:
    first_table = table_names[0]  # Change index to explore different tables
    sample_data = explore_table_structure(first_table)
    display(sample_data)


=== Table: events ===
Columns:
  event_id (int) - NO - PRI
  game_id (bigint) - NO - PRI
  period_number (int) - YES - 
  period_type (varchar(10)) - YES - 
  time_in_period (varchar(10)) - YES - 
  time_remaining (varchar(10)) - YES - 
  situation_code (varchar(10)) - YES - 
  home_team_defending_side (varchar(10)) - YES - 
  type_code (int) - YES - 
  type_desc_key (varchar(50)) - YES - 
  sort_order (int) - YES - 
  x_coord (int) - YES - 
  y_coord (int) - YES - 
  zone_code (varchar(5)) - YES - 
  shot_type (varchar(20)) - YES - 
  blocking_Player_id (int) - YES - 
  shooting_player_id (int) - YES - 
  goalie_in_net_id (int) - YES - 
  player_id (int) - YES - 
  event_owner_team_id (int) - YES - 
  away_sog (int) - YES - 
  home_sog (int) - YES - 
  hitting_player_id (int) - YES - 
  hittee_player_id (int) - YES - 
  reason (varchar(20)) - YES - 
  secondary_reason (varchar(20)) - YES - 
  losing_player_id (int) - YES - 
  winning_player_id (int) - YES - 

Total rows: 8,383,342

S

,event_id,game_id,period_number,period_type,time_in_period,time_remaining,situation_code,home_team_defending_side,type_code,type_desc_key,sort_order,x_coord,y_coord,zone_code,shot_type,blocking_Player_id,shooting_player_id,goalie_in_net_id,player_id,event_owner_team_id,away_sog,home_sog,hitting_player_id,hittee_player_id,reason,secondary_reason,losing_player_id,winning_player_id
0,2,2009020257,1,REG,00:16,19:44,1551,,504,giveaway,8,89,-32,D,,-1,-1,-1,8469623,30,-1,-1,-1,-1,,,-1,-1
1,2,2009020457,1,REG,00:00,20:00,-1,,520,period-start,5,-1,-1,,,-1,-1,-1,-1,-1,-1,-1,-1,-1,,,-1,-1
2,2,2009020482,1,REG,00:22,19:38,1551,,503,hit,9,92,27,D,,-1,-1,-1,-1,2,-1,-1,8468309,8462041,,,-1,-1
3,2,2009020709,1,REG,01:56,18:04,1551,,507,missed-shot,20,-51,-25,O,wrist,-1,8471657,8455710,-1,21,-1,-1,-1,-1,wide-of-net,,-1,-1
4,2,2010020090,1,REG,00:00,20:00,-1,,520,period-start,5,-1,-1,,,-1,-1,-1,-1,-1,-1,-1,-1,-1,,,-1,-1


## 4. Data Visualization

Create some quick visualizations to explore patterns in your data.

In [7]:
cursor.execute("""
    SELECT *
    FROM seasons_end_standings
""")

seasons_end_standings_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

cursor.execute("""
    SELECT *
    FROM teams
""")

teams = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

#Create an interactable table giving a season overview for the league with an option to choose which season

#adjust dataframe

# Rename columns to more user-friendly names
seasons_end_standings_df.rename(columns={
    'season_id': 'Season',
    'ot_losses': 'Overtime Losses',
    'games_played': 'Games Played',
    'points': 'Points',
    'wins': 'Wins',
    'losses': 'Losses'
}, inplace=True)

#Season values 12345678 -> 1234
seasons_end_standings_df['Season'] = seasons_end_standings_df['Season'].astype(str).str[:4]

available_seasons = seasons_end_standings_df['Season'].unique()

#Team -> get team_name from the teams df for each Team(formerly team_id)
# Merge the seasons_end_standings_df with the teams DataFrame to get team_name
seasons_end_standings_df = seasons_end_standings_df.merge(
    teams[['team_id', 'team_name']],  # Select only the relevant columns from teams
    left_on='team_id',                  # Match on the Team column in seasons_end_standings_df
    right_on='team_id',              # Match on the team_id column in teams
    how='left'                       # Use a left join to keep all rows from seasons_end_standings_df
)

# Drop the now redundant 'team_id' column
seasons_end_standings_df.drop(columns=['team_id'], inplace=True)

# Rename 'team_name' to 'Team' for clarity
seasons_end_standings_df.rename(columns={'team_name': 'Team'}, inplace=True)

columns = ['Team'] + [col for col in seasons_end_standings_df.columns if col != 'Team']
seasons_end_standings_df = seasons_end_standings_df[columns]

# Sort the DataFrame by the 'points' column in descending order
seasons_end_standings_df = seasons_end_standings_df.sort_values(by='Points', ascending=False)

# Select and reorder specific fields
selected_columns = ['Team', 'Season', 'Wins', 'Losses', 'Overtime Losses', 'Points', 'Games Played']
seasons_end_standings_df = seasons_end_standings_df[selected_columns]

# Create a dropdown widget for selecting a season
season_selector = widgets.Dropdown(
    options=available_seasons,
    description='Season:',
    value=available_seasons[0],
    style={'description_width': 'initial'}
)

# Function to update and display the table based on the selected season
def update_table(selected_season):
    filtered_df = seasons_end_standings_df[
        (seasons_end_standings_df['Season'] == selected_season) &
        (seasons_end_standings_df['Games Played'] > 0)
    ]
    display(filtered_df)  # Display the filtered DataFrame as a table

# Use the interact function to link the dropdown to the update_table function
widgets.interact(update_table, selected_season=season_selector)

interactive(children=(Dropdown(description='Season:', options=('1942', '1943', '1944', '1945', '1946', '1947',…

<function __main__.update_table(selected_season)>

In [ ]:
cursor.execute("""
    SELECT *
    FROM seasons_end_standings
    WHERE games_played > 0
""")

seasons_end_standings_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

cursor.execute("""
    SELECT *
    FROM teams
""")

teams = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

#adjust dataframe to be human readable

# Define a mapping of current column names to user-friendly titles
column_mapping = {
    'team_name': 'Team',
    'games_played': 'GP',
    'wins': 'W',
    'losses': 'L',
    'ot_losses': 'OTL',
    'points': 'PTS',
    'conference_name': 'Conference',
    'season_id': 'Season'
}

# Merge the seasons_end_standings_df with the teams DataFrame to get team_name
seasons_end_standings_df = seasons_end_standings_df.merge(
    teams[['team_id', 'team_name']],  # Select only the relevant columns from teams
    left_on='team_id',               # Match on the team_id column in seasons_end_standings_df
    right_on='team_id',              # Match on the team_id column in teams
    how='left'                       # Use a left join to keep all rows from seasons_end_standings_df
)


# Apply the mapping to rename the columns
seasons_end_standings_df.rename(columns=column_mapping, inplace=True)

seasons_end_standings_df = seasons_end_standings_df.sort_values(by='PTS', ascending=False)

available_seasons = sorted(seasons_end_standings_df['Season'].unique())

# Filter data by conference
eastern_conference_df = seasons_end_standings_df[seasons_end_standings_df['Conference'] == 'Eastern']
western_conference_df = seasons_end_standings_df[seasons_end_standings_df['Conference'] == 'Western']

# Choose the columns and the order to display
display_columns = ['Team', 'GP', 'W', 'L', 'OTL', 'PTS']


app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Example: Add a 'slug' column for team URLs
eastern_conference_df['slug'] = eastern_conference_df['Team'].str.replace(' ', '-').str.lower()
western_conference_df['slug'] = western_conference_df['Team'].str.replace(' ', '-').str.lower()

def make_table(df):
    rows = []
    for _, row in df.iterrows():
        team_link = dcc.Link(row['Team'], href=f"/team/{row['slug']}")
        cells = [html.Td(team_link)] + [html.Td(row[col]) for col in display_columns if col != 'Team']
        rows.append(html.Tr(cells))
    return dbc.Table(
        [html.Thead(html.Tr([html.Th(col) for col in display_columns]))] +
        [html.Tbody(rows)],
        striped=True, bordered=True, hover=True, responsive=True
    )

def standings_tables(df, selected_season):
    eastern = df[df['Conference'] == 'Eastern']
    western = df[df['Conference'] == 'Western']
    return dbc.Container([
        dbc.Row([
            dbc.Col(
                dcc.Dropdown(
                    id='season-dropdown',
                    options=[{'label': s, 'value': s} for s in available_seasons],
                    value=selected_season,
                    clearable=False,
                    style={'marginBottom': '1em'}
                ),
                width=12
            )
        ]),
        dbc.Row(dbc.Col(html.H1(f"NHL Standings {selected_season}-{int(selected_season)+1}", className="text-center my-4"), width=12)),
        dbc.Row([
            dbc.Col([html.H2("Eastern Conference", className="text-center"), make_table(eastern)], width=6),
            dbc.Col([html.H2("Western Conference", className="text-center"), make_table(western)], width=6)
        ])
    ], fluid=True)

def get_season_df(season):
    return seasons_end_standings_df[seasons_end_standings_df['Season'] == season]


app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='season-dropdown-container'),
    html.Div(id='page-content')
])

@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname'), Input('season-dropdown', 'value')]
)
def display_page(pathname, selected_season):
    season = selected_season if selected_season else available_seasons[0]
    df = get_season_df(season).copy()
    df['slug'] = df['Team'].str.replace(' ', '-').str.lower()
    if pathname and pathname.startswith('/team/'):
        team_slug = pathname.split('/team/')[1]
        team_row = df[df['slug'] == team_slug]
        if not team_row.empty:
            team_name = team_row.iloc[0]['Team']
            stats = [
                html.P(f"Games Played: {team_row.iloc[0]['GP']}"),
                html.P(f"Wins: {team_row.iloc[0]['W']}"),
                html.P(f"Losses: {team_row.iloc[0]['L']}"),
                html.P(f"OT Losses: {team_row.iloc[0]['OTL']}"),
                html.P(f"Points: {team_row.iloc[0]['PTS']}")
            ]
            return dbc.Container([
                html.A("← Back to Standings", href="/", style={"display": "block", "margin": "1em 0"}),
                html.H2(f"{team_name} Page"),
                *stats
            ], fluid=True)
        else:
            return dbc.Container("Team not found.", fluid=True)
    elif pathname == '/' or pathname == '':
        return html.Div([
            dcc.Dropdown(
                id='season-dropdown',
                options=[{'label': s, 'value': s} for s in available_seasons],
                value=season,
                clearable=False,
                style={'marginBottom': '1em'}
            ),
            standings_tables(df, season)
        ])
    else:
        return dbc.Container("Page not found.", fluid=True)



# Run the app
app.run(host='127.0.0.1', port=8050, debug=True)



## 5. Experimental Section

Use this section to write and test your own custom queries and analysis. This is your playground to tinker with the data!

In [ ]:
# Your experimental code goes here!
# Try different queries, create custom visualizations, or perform statistical analysis

# Example: Custom query
# my_query = """
# SELECT your_columns
# FROM your_table
# WHERE your_conditions
# """
# my_data = query_to_df(my_query)
# display(my_data)

# Example: Custom visualization
# your_viz_code_here

print("Ready for experimentation! Write your custom code above.")

## 6. Clean Up

Don't forget to close your database connection when you're done!

In [ ]:
# Close database connection
try:
    if cursor:
        cursor.close()
    if connection:
        connection.close()
    print("Database connection closed successfully")
except:
    print("No active connection to close")